In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
#参数随机初始化函数
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

#G(z)的平均先验
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [3]:
#为判别器准备的MNIST图像输入设置
X = tf.placeholder(tf.float32, shape=[None, 784])

#判别器参数设置
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))
D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
theta_D = [D_W1, D_W2, D_b1, D_b2]

#为生成器生成随机噪声
Z = tf.placeholder(tf.float32, shape=[None, 100])

#生成器参数设置
G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))
G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))
theta_G = [G_W1, G_W2, G_b1, G_b2]

In [4]:
#生成器网络
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

#判别器网络
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [5]:
#画图函数
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [8]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf
                         .log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=D_logit_real, logits=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=D_logit_fake, logits=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake
# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=D_logit_fake, logits=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())

    if not os.path.exists('../out/'):
        os.makedirs('../out/')

    i = 0

    for it in range(100000):
        if it % 1000 == 0:
            samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

            fig = plot(samples)
            plt.savefig('../out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
            i += 1
            plt.close(fig)

        X_mb, _ = mnist.train.next_batch(mb_size)

        _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
        _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})


        if it % 500 == 0:
            print('Iter: {}'.format(it))
            print('D loss: {:.4}'. format(D_loss_curr))
            print('G_loss: {:.4}'.format(G_loss_curr))
          #  plt.show(samples[0].reshape(28,28))
            print()
plt.rcParams.update({'figure.max_open_warning': 0})

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz
Iter: 0
D loss: 1.484
G_loss: 2.834

Iter: 500
D loss: 0.01453
G_loss: 7.156

Iter: 1000
D loss: 0.00399
G_loss: 8.379

Iter: 1500
D loss: 0.01679
G_loss: 6.924

Iter: 2000
D loss: 0.02795
G_loss: 5.495

Iter: 2500
D loss: 0.1015
G_loss: 5.507

Iter: 3000
D loss: 0.04562
G_loss: 4.877

Iter: 3500
D loss: 0.1454
G_loss: 5.489

Iter: 4000
D loss: 0.2633
G_loss: 4.586

Iter: 4500
D loss: 0.2231
G_loss: 4.598

Iter: 5000
D loss: 0.1578
G_loss: 5.203

Iter: 5500
D loss: 0.2406
G_loss: 5.98



/home/liujing/anaconda3/envs/yinpengyu/lib/python3.6/site-packages/matplotlib/image.py:395: UserWarning: Warning: converting a masked element to nan.
  dv = (np.float64(self.norm.vmax) -
/home/liujing/anaconda3/envs/yinpengyu/lib/python3.6/site-packages/matplotlib/image.py:396: UserWarning: Warning: converting a masked element to nan.
  np.float64(self.norm.vmin))
/home/liujing/anaconda3/envs/yinpengyu/lib/python3.6/site-packages/matplotlib/image.py:403: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
/home/liujing/anaconda3/envs/yinpengyu/lib/python3.6/site-packages/matplotlib/image.py:408: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)
/home/liujing/anaconda3/envs/yinpengyu/lib/python3.6/site-packages/matplotlib/colors.py:918: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
/home/liujing/anaconda3/envs/yinpengyu/lib/python3.6/site-packages/numpy/ma/core.py:718:

Iter: 6000
D loss: 1.389
G_loss: 0.7479

Iter: 6500
D loss: 1.386
G_loss: 0.6934

Iter: 7000
D loss: 1.386
G_loss: 0.6931

Iter: 7500
D loss: 1.386
G_loss: 0.6931

Iter: 8000
D loss: 1.386
G_loss: 0.6931

Iter: 8500
D loss: 1.386
G_loss: 0.6931

Iter: 9000
D loss: 1.386
G_loss: 0.6931

Iter: 9500
D loss: 1.386
G_loss: 0.6931



KeyboardInterrupt: 

In [ ]:
a = samples[0].reshape(28,28)

In [ ]:
plt.imshow(a)
plt.show()